Fetching the geolife data

In [171]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import read_geolife
import pois
import poi_query

In [172]:
traces = read_geolife.read_all_of_user("003")
trace = traces[7]
trace = trace[['Longitude','Latitude','Datetime']].to_numpy()
trace = [tuple(x) for x in trace]

print(len(trace))

2052


In [176]:
lonlat_clusters = {}
# get lon lat as list
lon = [x[0] for x in trace]  
lat = [x[1] for x in trace] 
lonlat  = [[lon, lat ] for lon, lat in zip(lon, lat)]
res = pois.cluster(lonlat, 25.0, 50)

for i, (cluster_type, cluster_id) in enumerate(res):
    if cluster_type != 'Noise':
        if cluster_id not in lonlat_clusters:
            lonlat_clusters[cluster_id] = {
                'points': []  
            }
        lonlat_clusters[cluster_id]['points'].append(trace[i])

#calculate avg of staypoint
for cluster_id, cluster_data in lonlat_clusters.items():
    points = cluster_data['points']
    if points:
        avg_lon = sum(point[0] for point in points) / len(points)
        avg_lat = sum(point[1] for point in points) / len(points)
        cluster_data['POI'] = poi_query.get_closest_poi(avg_lon, avg_lat)
        print(avg_lon,avg_lat)  
        print(cluster_data['POI'])     


116.31973311447811 40.00771256565657
Not found
116.32224333846153 40.00336333846154
Not found
116.32537678918918 39.99177949189189
Not found


In [174]:
fig = go.Figure()
for cluster_id, cluster_data in lonlat_clusters.items():
    #if cluster_data['POI'] != "Not found":
        lons = [point[0] for point in cluster_data['points']]
        lats = [point[1] for point in cluster_data['points']]
        datetime = [point[2].strftime('%Y-%m-%d %H:%M:%S') for point in cluster_data['points']]
        poi_name = cluster_data['POI']  

        fig.add_trace(go.Scattermapbox(
            lat=lats,
            lon=lons,
            mode='lines+markers',
            marker=go.scattermapbox.Marker(
                size=9,
            ),
            name=f'POI: '+ poi_name, 
            showlegend=True, 
            text=datetime
            ))

center_of_beijing = [39.9042, 116.4074]
fig.update_layout(mapbox_style="open-street-map",mapbox_center_lon=center_of_beijing[1], mapbox_center_lat=center_of_beijing[0],mapbox_zoom=10)

fig.show()

In [175]:
datetime = [c.strftime('%Y-%m-%d %H:%M:%S') for a, b, c in trace]  # format the datetime values
fig.add_trace(go.Scattermapbox(
    lat=[b for a,b,c in trace],
    lon=[a for a,b,c in trace],
    mode='lines+markers',
    marker=go.scattermapbox.Marker(
        size=9
    ),
    text=datetime
))